In [72]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [73]:
import pandas as pd
import numpy as np
import copy

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [74]:
TARGET_NAME = 'units'

#### Считывание данных

In [75]:
key = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/key.csv")
train = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/train.csv")
test = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/test.csv")
weather = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/weather.csv")

#### Описание данных

In [76]:
#### Предобработка данных

In [77]:
def minutes_from_midnight(time_str):
    if len(time_str) == 1:
        return np.nan
    return int(time_str[:2])*60 + int(time_str[2:])

In [78]:
weather.sample()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
19771,15,2014-09-24,68,39,54,-2,45,49,11,0,0535,1740,FG+ FG BR,0.0,0.00,30.19,30.59,1.6,12,2.1


In [79]:
weather.wetbulb.value_counts()

M      1252
74      777
75      684
73      663
72      609
71      526
70      481
67      474
65      459
64      437
62      433
68      428
66      425
63      423
60      419
61      409
59      408
58      406
69      399
56      383
76      362
55      356
57      344
52      337
54      330
53      325
44      323
45      307
50      305
51      304
       ... 
17       54
15       53
16       48
12       43
10       40
13       38
11       33
8        31
79       21
4        20
7        16
9        14
0        14
5        13
3        11
6         9
2         7
-3        7
1         5
-4        4
-2        3
-5        3
-6        2
-7        2
-10       2
-15       1
-8        1
-9        1
-1        1
80        1
Name: wetbulb, Length: 93, dtype: int64

#### Предобработка данных

In [80]:
def preprocessing(data, key, weather, target_field=TARGET_NAME):
    data = data.join(key.set_index("store_nbr"), on="store_nbr")
    data = data.join(weather.set_index(["station_nbr", "date"]), on=["station_nbr", "date"])
    
    data['sunrise'] = data['sunrise'].apply(minutes_from_midnight)
    data['sunrise'] = data['sunset'].apply(minutes_from_midnight)
#     data.drop(columns=["codesum"])
    data.fillna(-1, inplace=True)
    
    needed_columns = ["store_nbr", "item_nbr"]
    if target_field in data.columns:
        needed_columns.append(target_field)
    
    data = data[needed_columns]
    data = pd.get_dummies(data, columns=["store_nbr", "item_nbr"])
    if target_field in data.columns:
        data[target_field] = np.log(data[target_field] + 1)
        
    return data

In [81]:
preprocessed_train = preprocessing(train, key, weather)
preprocessed_test = preprocessing(test, key, weather)

train_columns = preprocessed_train.columns
test_columns = preprocessed_test.columns
diff_columns = list(set(train_columns) - set(test_columns))
diff_columns.remove(TARGET_NAME)
print("Diff of columns {}". format(diff_columns))
for column in diff_columns:
    preprocessed_test[column] = 0 # Add missing field after onehot encoding

Diff of columns ['store_nbr_35']


# Обучение базовой модели

In [82]:
y = preprocessed_train['units']
X = preprocessed_train.loc[:, preprocessed_train.columns != 'units']

In [84]:
model = RandomForestRegressor(criterion='mse')

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

/Users/skrrydg/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [86]:
X_train = X_train[0:100000]
y_train = y_train[0:100000]

In [87]:
model.fit(X_train, y_train)

/Users/skrrydg/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [88]:
y_predict = model.predict(X_test)

In [89]:
def get_score(y_true, y_predict):
    return math.sqrt(mean_squared_error(y_true, y_predict))

In [90]:
get_score(y_test, y_predict)

0.13923403081521057

#### Предсказание

In [91]:
y_predict = model.predict(preprocessed_test)

In [96]:
y_predict = np.exp(y_predict) - 1

In [99]:
print(res[res > 0])

[ 59.63862717   0.57169382 111.36161867 ...  44.44717473   0.47657575
   0.92670168]


In [112]:
test.head()

,date,store_nbr,item_nbr
0,2013-04-01,2,1
1,2013-04-01,2,2
2,2013-04-01,2,3
3,2013-04-01,2,4
4,2013-04-01,2,5


In [118]:
result = pd.DataFrame

In [133]:
_id = (test["store_nbr"].astype("str") + "_" + test["item_nbr"].astype("str") + "_" + test["date"].astype('str')).to_numpy()
_units = np.round(y_predict).astype(int)

In [141]:
prediction = pd.DataFrame(data={"id": _id, "units": _units}).set_index("id") 

In [143]:
prediction.to_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/submissions/benchmark1.csv")

In [115]:
test["store_nbr"].astype("str") + "_" + test["item_nbr"].astype("str") + "_" + test["date"].astype('str')

0            2_1_2013-04-01
1            2_2_2013-04-01
2            2_3_2013-04-01
3            2_4_2013-04-01
4            2_5_2013-04-01
5            2_6_2013-04-01
6            2_7_2013-04-01
7            2_8_2013-04-01
8            2_9_2013-04-01
9           2_10_2013-04-01
10          2_11_2013-04-01
11          2_12_2013-04-01
12          2_13_2013-04-01
13          2_14_2013-04-01
14          2_15_2013-04-01
15          2_16_2013-04-01
16          2_17_2013-04-01
17          2_18_2013-04-01
18          2_19_2013-04-01
19          2_20_2013-04-01
20          2_21_2013-04-01
21          2_22_2013-04-01
22          2_23_2013-04-01
23          2_24_2013-04-01
24          2_25_2013-04-01
25          2_26_2013-04-01
26          2_27_2013-04-01
27          2_28_2013-04-01
28          2_29_2013-04-01
29          2_30_2013-04-01
                ...        
526887     45_82_2014-10-26
526888     45_83_2014-10-26
526889     45_84_2014-10-26
526890     45_85_2014-10-26
526891     45_86_201